In [23]:
import pandas as pd
import numpy as np
from embedding import get_embedding

In [84]:
data = {'ESTIMATE': ["I think [AGENT] is a [ROLE]", "[AGENT] must be the [ROLE]", "[AGENT] seems like the [ROLE]"],
                          'CO': ["I am a [ROLE]", "My role is the [ROLE]"], 
                          'SUSPECT': ["I can't believe [AGENT]", "[AGENT] is suspicious", "[AGENT] is lying"], 
                          'VOTE': ["I will vote for [AGENT]", "My choice is [AGENT]"], 
                          'DIVINATION': ["I will divine [AGENT]", "I might investigate [AGENT]"], 
                          'DIVINED': ["I divined [AGENT] as a [TEAM]", "[AGENT] was divined as a [TEAM]", "[AGENT] is determined to be [TEAM]"], 
                          'AGREE': ["I agree with [AGENT]", "What [AGENT] said is correct"], 
                          'NOT ESTIMATE': ["I don't think [AGENT] is a [ROLE]", "[AGENT] is not [ROLE]", "[AGENT] doesn't seem like the [ROLE]"],
                          'NOT CO': ["I am not a [ROLE]", "It can't be true that I am the [ROLE]"], 
                          'NOT SUSPECT': ["I don't suspect [AGENT]", "I don't think [AGENT] is lying"], 
                          'NOT VOTE': ["I won't vote for [AGENT]", "I disagree with voting for [AGENT]"], 
                          'NOT AGREE': ["I disagree with [AGENT]", "What [AGENT] said is wrong"], 
                          }

In [85]:
df = pd.DataFrame.from_dict(data, orient='index').T # これだとうまくいく
df

,ESTIMATE,CO,SUSPECT,VOTE,DIVINATION,DIVINED,AGREE,NOT ESTIMATE,NOT CO,NOT SUSPECT,NOT VOTE,NOT AGREE
0,I think [AGENT] is a [ROLE],I am a [ROLE],I can't believe [AGENT],I will vote for [AGENT],I will divine [AGENT],I divined [AGENT] as a [TEAM],I agree with [AGENT],I don't think [AGENT] is a [ROLE],I am not a [ROLE],I don't suspect [AGENT],I won't vote for [AGENT],I disagree with [AGENT]
1,[AGENT] must be the [ROLE],My role is the [ROLE],[AGENT] is suspicious,My choice is [AGENT],I might investigate [AGENT],[AGENT] was divined as a [TEAM],What [AGENT] said is correct,[AGENT] is not [ROLE],It can't be true that I am the [ROLE],I don't think [AGENT] is lying,I disagree with voting for [AGENT],What [AGENT] said is wrong
2,[AGENT] seems like the [ROLE],None,[AGENT] is lying,None,None,[AGENT] is determined to be [TEAM],None,[AGENT] doesn't seem like the [ROLE],None,None,None,None


In [86]:
df.fillna(np.nan, inplace=True)
df

,ESTIMATE,CO,SUSPECT,VOTE,DIVINATION,DIVINED,AGREE,NOT ESTIMATE,NOT CO,NOT SUSPECT,NOT VOTE,NOT AGREE
0,I think [AGENT] is a [ROLE],I am a [ROLE],I can't believe [AGENT],I will vote for [AGENT],I will divine [AGENT],I divined [AGENT] as a [TEAM],I agree with [AGENT],I don't think [AGENT] is a [ROLE],I am not a [ROLE],I don't suspect [AGENT],I won't vote for [AGENT],I disagree with [AGENT]
1,[AGENT] must be the [ROLE],My role is the [ROLE],[AGENT] is suspicious,My choice is [AGENT],I might investigate [AGENT],[AGENT] was divined as a [TEAM],What [AGENT] said is correct,[AGENT] is not [ROLE],It can't be true that I am the [ROLE],I don't think [AGENT] is lying,I disagree with voting for [AGENT],What [AGENT] said is wrong
2,[AGENT] seems like the [ROLE],NaN,[AGENT] is lying,NaN,NaN,[AGENT] is determined to be [TEAM],NaN,[AGENT] doesn't seem like the [ROLE],NaN,NaN,NaN,NaN


In [111]:
class MakeDataframe:
    def __init__(self):
        self.datalist = None

    def make_data(self, df):
        """データフレームを受け取り、[AGENT]と[ROLE]と[TEAM]を全ての組み合わせで埋める"""
        Agents = ["Agent[00]", "Agent[01]", "Agent[02]", "Agent[03]", "Agent[04]"]
        Roles = ["VILLAGER", "SEER", "WEREWOLF", "POSSESSED"]
        Teams = ["BLACK", "WHITE", "VILLAGER", "HUMAN", "WEREWOLF"] # 精度のためにteamsに"VILLAGER", "WEREWOLF"も追加
        
        self.datalist = []
        for column in df.columns: 
            original_sentences = df[column].dropna().values 
            for original_sentence in original_sentences:
                if "[AGENT]" in original_sentence and "[ROLE]" in original_sentence:
                    for x in Agents:
                        for y in Roles:
                            ans = {} # 各sentenceの情報を入れる辞書
                            ans["sentence"] = original_sentence.replace('[AGENT]', x).replace('[ROLE]', y)
                            ans["Verb"] = column
                            ans["Agent"] = x
                            ans["Role"] = y
                            ans["Team"] = None
                            self.datalist.append(ans)
                elif "[AGENT]" in original_sentence and "[TEAM]" in original_sentence:
                    for x in Agents:
                        for y in Teams:
                            ans = {} # 各sentenceの情報を入れる辞書
                            ans["sentence"] = original_sentence.replace('[AGENT]', x).replace('[TEAM]', y)
                            ans["Verb"] = column
                            ans["Agent"] = x
                            ans["Role"] = None
                            ans["Team"] = y
                            self.datalist.append(ans)
                elif "[AGENT]" in original_sentence:
                    for x in Agents:
                        ans = {} # 各sentenceの情報を入れる辞書
                        ans["sentence"] = original_sentence.replace('[AGENT]', x)
                        ans["Verb"] = column
                        ans["Agent"] = x
                        ans["Role"] = None
                        ans["Team"] = None
                        self.datalist.append(ans)
                elif '[ROLE]' in original_sentence:
                    for y in Roles:
                        ans = {} # 各sentenceの情報を入れる辞書
                        ans["sentence"] = original_sentence.replace('[ROLE]', y)
                        ans["Verb"] = column
                        ans["Agent"] = None
                        ans["Role"] = y
                        ans["Team"] = None
                        self.datalist.append(ans)
        return self.datalist, len(self.datalist)
    def make_dataframe(self, df):
        self.make_data(df)
        df = pd.DataFrame(self.datalist)   
        df["Team"] = df["Team"].replace({"VILLAGER": "WHITE", "HUMAN": "WHITE", "WEREWOLF": "BLACK"})
        return df
    def dataframe_embed(self, df):
        df = self.make_dataframe(df)
        df["embedded"] = df["sentence"].map(lambda x: get_embedding(x))
        # リストに変換
        df["embedded"] = df["embedded"].map(lambda x: list(x))
        df.to_csv("embedded.csv")
        return df
a = MakeDataframe()

In [112]:
a.make_data(df)

([{'sentence': 'I think Agent[00] is a VILLAGER',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'VILLAGER',
   'Team': None},
  {'sentence': 'I think Agent[00] is a SEER',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'SEER',
   'Team': None},
  {'sentence': 'I think Agent[00] is a WEREWOLF',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'WEREWOLF',
   'Team': None},
  {'sentence': 'I think Agent[00] is a POSSESSED',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'POSSESSED',
   'Team': None},
  {'sentence': 'I think Agent[01] is a VILLAGER',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'VILLAGER',
   'Team': None},
  {'sentence': 'I think Agent[01] is a SEER',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'SEER',
   'Team': None},
  {'sentence': 'I think Agent[01] is a WEREWOLF',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'WEREWOLF',
   'Team': None},
  {'sentence': 'I think Agent[01] is a PO

In [113]:
a.make_dataframe(df)

,sentence,Verb,Agent,Role,Team
0,I think Agent[00] is a VILLAGER,ESTIMATE,Agent[00],VILLAGER,None
1,I think Agent[00] is a SEER,ESTIMATE,Agent[00],SEER,None
2,I think Agent[00] is a WEREWOLF,ESTIMATE,Agent[00],WEREWOLF,None
3,I think Agent[00] is a POSSESSED,ESTIMATE,Agent[00],POSSESSED,None
4,I think Agent[01] is a VILLAGER,ESTIMATE,Agent[01],VILLAGER,None
...,...,...,...,...,...
281,What Agent[00] said is wrong,NOT AGREE,Agent[00],None,None
282,What Agent[01] said is wrong,NOT AGREE,Agent[01],None,None
283,What Agent[02] said is wrong,NOT AGREE,Agent[02],None,None
284,What Agent[03] said is wrong,NOT AGREE,Agent[03],None,None


In [114]:
a.dataframe_embed(df)

,sentence,Verb,Agent,Role,Team,embedded
0,I think Agent[00] is a VILLAGER,ESTIMATE,Agent[00],VILLAGER,None,"[-0.03843538090586662, 0.0017339562764391303, ..."
1,I think Agent[00] is a SEER,ESTIMATE,Agent[00],SEER,None,"[-0.02783491648733616, -0.012529968284070492, ..."
2,I think Agent[00] is a WEREWOLF,ESTIMATE,Agent[00],WEREWOLF,None,"[-0.01916315034031868, 0.0025325417518615723, ..."
3,I think Agent[00] is a POSSESSED,ESTIMATE,Agent[00],POSSESSED,None,"[-0.03817843645811081, 0.024196751415729523, -..."
4,I think Agent[01] is a VILLAGER,ESTIMATE,Agent[01],VILLAGER,None,"[-0.040945056825876236, 0.006486270576715469, ..."
...,...,...,...,...,...,...
281,What Agent[00] said is wrong,NOT AGREE,Agent[00],None,None,"[-0.03842293843626976, 0.019114114344120026, 0..."
282,What Agent[01] said is wrong,NOT AGREE,Agent[01],None,None,"[-0.03957590460777283, 0.01657181605696678, 0...."
283,What Agent[02] said is wrong,NOT AGREE,Agent[02],None,None,"[-0.032267309725284576, 0.016541307792067528, ..."
284,What Agent[03] said is wrong,NOT AGREE,Agent[03],None,None,"[-0.036613430827856064, 0.01735736057162285, 0..."
